In [1]:
# https://www.tensorflow.org/tutorials/quickstart/beginner?hl=es-419
import tensorflow as tf

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
y_train.shape

(60000,)

# Guardar MNIST en disco

In [8]:
# Guardar imágenes en disco y crear csvs para tener las mismas condiciones
import pandas as pd
import numpy as np
import cv2
import os

data_path = 'MNIST_dataset/'

if not os.path.exists(data_path):
    os.mkdir(data_path)

    paths = []
    labels = []

    num_images = 0
    for i in range(x_train.shape[0]):
        paths.append(data_path + str(num_images).zfill(5) + '.png')
        labels.append(int(y_train[i]))

        cv2.imwrite(data_path + str(num_images).zfill(5) + '.png', x_train[i,...])
        num_images += 1

    csv = pd.DataFrame({
        'path': paths,
        'DR_level': labels
    })
    csv.to_csv('MNIST_train.csv', index=False)

    paths = []
    labels = []

    for i in range(x_test.shape[0]):
        paths.append(data_path + str(num_images).zfill(5) + '.png')
        labels.append(int(y_test[i]))

        cv2.imwrite(data_path + str(num_images).zfill(5) + '.png', x_test[i,...])
        num_images += 1

    csv = pd.DataFrame({
        'path': paths,
        'DR_level': labels
    })
    csv.to_csv('MNIST_val.csv', index=False)

In [9]:
# Borrar datasets descargados de memoria
del x_train, y_train, x_test, y_test

# Codigo propio

In [1]:
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[i] for i in range(10)]

# Specify dataset files
TRAIN_FILE = 'MNIST_train.csv'
VALIDATION_FILE = 'MNIST_val.csv'

TRAINING_BATCH_SIZE = 32
TRAINING_DATA_AUG = False
TRAINING_BALANCED = False
TRAINING_SHUFFLE = False
TRAINING_PREFETCH = None
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 32
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_SHUFFLE = False
VALIDATION_PREFETCH = None
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    shuffle=TRAINING_SHUFFLE,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                shuffle=VALIDATION_SHUFFLE, 
                                                size=VALIDATION_TAKE_SIZE)

In [5]:
for img, lb in train_dataset.take(4):
    print(img.shape)
    print(lb.shape)
    print('----')

(32, 28, 28, 3)
(32, 10)
----
(32, 28, 28, 3)
(32, 10)
----
(32, 28, 28, 3)
(32, 10)
----
(32, 28, 28, 3)
(32, 10)
----


In [19]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 3)),
  tf.keras.layers.Dense(48, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

cbacks = [tf.keras.callbacks.TensorBoard('logs/MNIST_codigo_propio', histogram_freq=1, write_graph=False)]

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
# See Tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 47036), started 2:21:41 ago. (Use '!kill 47036' to kill it.)

In [22]:
model.fit(train_dataset, epochs=50, validation_data=val_dataset, verbose=1, callbacks=cbacks)

Epoch 1/50
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3838 - accuracy: 0.8845 - val_loss: 0.1800 - val_accuracy: 0.9471
Epoch 2/50
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2488 - accuracy: 0.9261 - val_loss: 0.1460 - val_accuracy: 0.9565
Epoch 3/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2117 - accuracy: 0.9352 - val_loss: 0.1374 - val_accuracy: 0.9594
Epoch 4/50
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1922 - accuracy: 0.9416 - val_loss: 0.1209 - val_accuracy: 0.9660
Epoch 5/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1789 - accuracy: 0.9451 - val_loss: 0.1253 - val_accuracy: 0.9638
Epoch 6/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1717 - accuracy: 0.9471 - val_loss: 0.1164 - val_accuracy: 0.9675
Epoch 7/50
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1586 - accuracy: 0.9496 - val_loss: 0.1123 - val_accuracy: